# CS591S1 - Differential Privacy - Spring 2021

## Assignment 2

This notebook provides the solution to Exercise 2 of Assignment #2. This includes the algorithms for the Report Noisy Max version of the Median Algorithm.

In [1]:
import numpy as np
from numpy import linalg as LA
import random
import matplotlib.pyplot as plt

In [41]:
def init(n, R, setting, k):
    li = []
    for i in range(n):
        if setting == 1:
            li.append(np.random.normal(R/4, (R*R)/10))
        elif setting == 2:
            li.append(np.random.poisson(50))
        elif setting == 3:
            li.append(np.random.uniform((R - k), (R + k)))
        else: # debugging setting: some stupid uniform distribution
            li.append(np.random.uniform(1, R))
    return np.array(li)

In [42]:
x = init(10, 100, 1, 0)
print(x)

[ 1185.41011506  1352.73836258 -1265.70571055   286.73260964
    24.30862773    24.76039264   687.31725995  1518.32828886
  -819.7180702   -383.73213602]


In [27]:
def round_to_nearest(x):
    li = []
    for i in range(len(x)):
        li.append(round(x[i]))
    return np.array(li)

In [28]:
rounded_x = round_to_nearest(x)
print(rounded_x)

[60. 76. 74. 17. 81. 58. 43. 55. 20. 90.]


In [29]:
def sign(z):
    if z > 0:
        return 1
    elif z < 0:
        return -1
    else:
        return 0

def score_func(y, x):
    q = 0
    for i in range(len(x)):
        q += sign(y - x[i])
    return -abs(q)

In [30]:
q = score_func(31, rounded_x)
print(q)

-6


In [105]:
def rnm_median(x, R, epsilon):
    rounded_x = round_to_nearest(x)
    li = []
    for y in range(1, R+1):
        Z_y = np.random.exponential(2/epsilon)
        q = score_func(y, x)
        noisy_q = q + Z_y
        li.append(noisy_q)
    return li.index(max(li)) + 1

In [112]:
# determine the position of y if it is inserted into the data array x
def rank(y, x):
    sorted_x = x.sort()
    i = len(x) - 1
    while i >= 0 and x[i] > y: 
        i -= 1 
    return (i + 2) 

In [113]:
x = init(10, 100, 2, 0)
print(x)
y_hat = rnm_median(x, 100, 0.1)
r = rank(y_hat, x)
#print(y_hat)
print(str(y_hat) + ", " + str(r))

[35 53 34 50 49 60 65 48 39 43]
47, 5


In [123]:
setting = 2
n_list = [500]
#R_list = [1000]
#n_list = [50, 100, 500, 2000, 10000]
R_list = [100, 1000, 10000]
output_data = []
for n in n_list:
    for R in R_list:
        print("n = " + str(n) + ", R = " + str(R))
        for i in range(0, 50):
            x = init(n, R, setting, 0)
            datum = []
            for j in range(0, 10):
                y_hat = rnm_median(x, R, 0.1)
                datum.append(abs(n/2 - rank(y_hat, x)))
            output_data.append(datum)
        #print(output_data)
        flatten_data = np.array(output_data).flatten()
        print("Average error in rank: " + str(np.mean(flatten_data)))
        print("Std of error in rank: " + str(np.std(flatten_data)))
        stds = []
        for k in range(len(output_data)):
            stds.append(np.std(output_data[k]))
        print("Std of error in rank among runs: " + str(np.mean(stds)))
        output_data = []
        print("-----------------------------------------")

n = 500, R = 100
Average error in rank: 16.342
Std of error in rank: 10.898671295162544
Std of error in rank among runs: 7.246200785219552
-----------------------------------------
n = 500, R = 1000
Average error in rank: 17.014
Std of error in rank: 9.672321541388087
Std of error in rank among runs: 6.267183080283198
-----------------------------------------
n = 500, R = 10000
Average error in rank: 17.744
Std of error in rank: 12.02191598706296
Std of error in rank among runs: 9.03522665743871
-----------------------------------------
